# Use R Shiny app to create SIR model with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate support of external R Shiny application code with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. 

Some familiarity with Python is helpful. This notebook uses Python 3.

## Learning goals

The learning goals of this notebook are:

-  Persist a R Shiny app in in Watson Machine Learning asset repository.
-  Deploy application for online scoring using client library.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Application upload](#upload)
3.	[Web service creation](#deploy)
4.  [Score](#score)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.0'
}

Alternatively you can use `username` and `password` to authenticate WML services.

```
wml_credentials = {
    "username": ***,
    "password": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.0'
}

```

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Application upload

In this section you will learn how to store downloaded application
as a WML asset.

#### Download R Shiny app from git project using wget
**Hint**: To install `wget` execute `!pip install wget`.

In [10]:
import os
import wget

filename = "app.R.zip"
url = "https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/r_shiny/app.R.zip"
if not os.path.isfile(filename):
    filename = wget.download(url)

#### Upload application as data asset

In [35]:
shiny_details = client.shiny.store(
    meta_props={
        client.shiny.ConfigurationMetaNames.NAME: "R Shiny app - SIR",
    },
    file_path=filename
)
shiny_asset_id = client.shiny.get_id(shiny_details)

Creating Shiny asset...
SUCCESS


**Note:** You can see that application is saved in Watson Machine Learning Service.

In [ ]:
client.shiny.list(limit=5)

<a id="deploy"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored application (web service).

In [41]:
deployment = client.deployments.create(
    artifact_uid=shiny_asset_id,
    meta_props={
                client.deployments.ConfigurationMetaNames.NAME: "Deployment R Shiny's SIR",
                client.deployments.ConfigurationMetaNames.R_SHINY: {"authentication" : "anyone_with_url" },
                client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name":"S", "num_nodes":1},
    }
)
deployment_id = client.deployments.get_id(deployment)



#######################################################################################

Synchronous deployment creation for uid: '0f73fdef-1fea-46c7-96b0-7f91208b931c' started

#######################################################################################


initializing.................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2c85fe90-b624-4546-adbb-d4a2408db20a'
------------------------------------------------------------------------------------------------




<a id="score"></a>
#### Get deployments details

In [42]:
deployment_id = client.deployments.get_id(deployment)
client.deployments.get_details(deployment_id)

{'entity': {'asset': {'id': '0f73fdef-1fea-46c7-96b0-7f91208b931c'},
  'custom': {},
  'deployed_asset_type': 'r_shiny',
  'description': 'deployment rshiny deployment',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'deployment_rshiny',
  'r_shiny': {'authentication': 'anyone_with_url'},
  'space_id': '757f4731-bb33-48d9-a871-3fad5c16c954',
  'status': {'rshiny_url': {'url': 'https://wmlgmc-cpd-wmlgmc.apps.wmlautoai.cp.fyre.ibm.com/ml/v4/deployments/2c85fe90-b624-4546-adbb-d4a2408db20a/r_shiny'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-02-16T12:12:50.695Z',
  'description': 'deployment rshiny deployment',
  'id': '2c85fe90-b624-4546-adbb-d4a2408db20a',
  'modified_at': '2021-02-16T12:12:50.695Z',
  'name': 'deployment_rshiny',
  'owner': '1000330999',
  'space_id': '757f4731-bb33-48d9-a871-3fad5c16c954'}}

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. 
 
 Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.